# Parcial de Regresión Lineal


## Preguntas de Negocio
1. **Predicción:** Si llega un carro con 92670.5 km, ¿a cuánto debería venderlo según el modelo?
2. **Inversión:** Como inversionista, ¿cuáles son los vehículos (según su kilometraje) que podrían generar un margen de ganancia atractivo?


In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

%matplotlib inline

## 1. Cargar y Preprocesar Datos

Se descarga el dataset de vehículos, se descomprime y se carga en un DataFrame. Además, se eliminan los valores faltantes.

In [2]:
# Descargar y descomprimir el dataset
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
!unzip -o cars_dataset.zip


--2025-03-12 20:21:12--  https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip [following]
--2025-03-12 20:21:12--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19753181 (19M) [application/zip]
Saving to: ‘cars_dataset.zip’

cars_dataset.zip    100%[===================>]  18.84M  --.-KB/s    in 0.1s    

2025-03-12 20:21:12 (133 MB/s) - ‘cars_dataset.zip’ saved [19753181/

In [4]:
pd.read_csv('car_prices.csv')

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558832,2015,Kia,K900,Luxury,Sedan,NaN,knalw4d4xf6019304,in,45.0,18255.0,silver,black,avis corporation,35300.0,33000.0,Thu Jul 09 2015 07:00:00 GMT-0700 (PDT)
558833,2012,Ram,2500,Power Wagon,Crew Cab,automatic,3c6td5et6cg112407,wa,5.0,54393.0,white,black,i -5 uhlmann rv,30200.0,30800.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558834,2012,BMW,X5,xDrive35d,SUV,automatic,5uxzw0c58cl668465,ca,48.0,50561.0,black,black,financial services remarketing (lease),29800.0,34000.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558835,2015,Nissan,Altima,2.5 S,sedan,automatic,1n4al3ap0fc216050,ga,38.0,16658.0,white,black,enterprise vehicle exchange / tra / rental / t...,15100.0,11100.0,Thu Jul 09 2015 06:45:00 GMT-0700 (PDT)


In [5]:
df=pd.read_csv('car_prices.csv')

In [6]:
df.isnull().sum()

,0
year,0
make,10301
model,10399
trim,10651
body,13195
transmission,65352
vin,4
state,0
condition,11820
odometer,94


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


## 2. Análisis Exploratorio de Datos (EDA)

Se generan estadísticas descriptivas para analizar las variables numéricas y entender la distribución de los datos.

In [7]:
df.describe()

,year,condition,odometer,mmr,sellingprice
count,558837.000000,547017.000000,558743.000000,558799.000000,558825.000000
mean,2010.038927,30.672365,68320.017767,13769.377495,13611.358810
std,3.966864,13.402832,53398.542821,9679.967174,9749.501628
min,1982.000000,1.000000,1.000000,25.000000,1.000000
25%,2007.000000,23.000000,28371.000000,7100.000000,6900.000000
50%,2012.000000,35.000000,52254.000000,12250.000000,12100.000000
75%,2013.000000,42.000000,99109.000000,18300.000000,18200.000000
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000


## 3. Modelo de Regresión Lineal

Se utiliza la variable 'odometer' para predecir 'sellingprice'. Se dividen los datos en conjunto de entrenamiento y prueba (80%-20%).

In [49]:
X = df[["odometer"]]
y = df["sellingprice"]

In [50]:
df['odometer'].fillna(df['odometer'].mean(),inplace=True)

<ipython-input-50-76837bd5a3dc>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['odometer'].fillna(df['odometer'].mean(),inplace=True)


In [51]:
df['sellingprice'].fillna(df['sellingprice'].mean(),inplace=True)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(f"Entrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")

Entrenamiento: 447069 muestras
Prueba: 111768 muestras


## 4. Evaluación del Modelo

Se evalúa el desempeño del modelo en el conjunto de prueba utilizando métricas como MSE, MAE y R².

In [53]:
model = LinearRegression()

In [54]:
model.fit(X_train, y_train)

LinearRegression()

In [55]:
intercept = model.intercept_
coef = model.coef_[0]

print(f"Intercept (beta_0): {intercept}")
print(f"Coeficiente (beta_1): {coef}")

Intercept (beta_0): 20860.126041486998
Coeficiente (beta_1): -0.1060274463931155


In [57]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [58]:
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mae_train= mean_absolute_error(y_train, y_pred_train)

In [59]:
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

In [60]:
print("Conjunto de Entrenamiento:")
print(f"MSE: {mse_train}")
print(f"R^2: {r2_train}")
print(f"MAE: {mae_train}")
print("\nConjunto de Prueba:")
print(f"MSE: {mse_test}")
print(f"R^2: {r2_test}")
print(f"MAE: {mae_test}")

Conjunto de Entrenamiento:
MSE: 63225314.017707594
R^2: 0.33717791129301444
MAE: 5496.867602724755

Conjunto de Prueba:
MSE: 61195294.400367975
R^2: 0.3469042956275835
MAE: 5461.051319817094


## 5. Predicción y Respuestas a Preguntas de Negocio

### Pregunta 1: Precio Estimado para un Carro con 92670.5 km

Utilizamos el modelo para predecir el precio de venta de un carro con 92670.5 km.

In [ ]:
# Predicción para un carro con 92670.5 km
#nuevo_odometro = np.array([[92670.5]])

In [61]:
nuevo_odometro = np.array([[92670.5]])

In [62]:
precio_predicho = model.predict(nuevo_odometro)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [63]:
print(f"El precio estimado para un carro con 92670.5 km es: {precio_predicho[0]}")

El precio estimado para un carro con 92670.5 km es: 11034.509570513788


## 6. Análisis Crítico de Resultados (10 Puntos Cerrados)

Responde a las siguientes preguntas de manera cerrada (Sí/No). Estas preguntas facilitan una calificación rápida:

1. ¿El coeficiente negativo (-0.107) indica que a mayor kilometraje, menor precio de venta? (Sí)


Rta: Primero pues es lógica que un carro a mayor kilometraje pues menor es su valor, pero pues como para dar una respuesta más técnica, sabemos que el coeficiente en un modelo así representa la pendiente, y al ser negativo vemos que esta baja cada ves que la otra variable sube.

2. ¿Un R² de 0.336 sugiere que el modelo explica más del 50% de la variabilidad en el precio de venta? (No)

Rta: Porque como tal ese R2 es la variabilidad del precio, y pues ese valor, representa un 33,6% que es menor al 50% de la misma.

3. ¿El valor elevado del MSE indica que existen errores importantes en las predicciones? (Sí)

Rta: Porque como tal el MSE es el Error Cudrático Medio, y pues entre más grande, mayor probabilidad de que las predicciones tengan errores.

4. ¿El MSE obtenido es suficientemente bajo para garantizar predicciones precisas en todos los casos? (No)

Rta: Porque el MSE es elevado, y puede indicar que el modelo no esté capturando bien la relación entre el kilometraje y el precio del carro, o que hay otros factores importantes que no se están considerando en el modelo.

5. ¿La eliminación de filas con valores faltantes puede reducir la robustez del modelo? (Sí)

Rta: Ya que la robustez la podemos entender como la capacidad del modelo para adaptarse, al quitarle esas filas podríamos quitarle oportunidades de aprender y centrarlo en que se bueno en una sola cosa.

6. ¿Incluir variables adicionales como año, condición y marca podría mejorar la capacidad predictiva del modelo? (Sí)

Rta: Ya que en el mundo automotriz, estos son factores bastante importantes a la hora de analizar el precio de un auto, el auto entre más "nuevo modelo" sea, mayor es su precio, la marca ya que hay marcas caras que son sinónimo de calidad o status, o la condición ya que uno usado o con algún golpe tiende a depreciarse; por lo que incluir estas variables en el modelo puede ayudarle a aprender patrones más complejos o realizar predicciones más precisas.

7. ¿La relación negativa entre kilometraje y precio es consistente con el comportamiento esperado en el mercado de vehículos? (Sí)

Rta: Ya que la relación negativa entre kilometraje y precio es consistente con el comportamiento esperado en el mercado de vehículos. A mayor kilometraje, menor precio, debido a la depreciación y la menor demanda de carros con mayor uso.

8. ¿Dividir los datos en 80% entrenamiento y 20% prueba es una práctica recomendada para evaluar el modelo? (Sí)

Rta: Pues porque al tener una base tan gruesa de entrenamiento, podemos hacer más efectivo nuestro modelo y enseñarle mejor las cosas que debe aprender por medio de los datos, y el restante sería para poder confirmar que sí haya aprendido de manera correcta.
